# Объединение dataframe

Структура шага

* аудио по join и union
    * под него слайды Добавление строк и/или колонок, Join() и их виды, Метод join() и его параметры, Union() и его использование
* аудио про lazy evaluation
    * под него слайд Lazy evaluation и оптимизация
* все остальное - материалы

### Добавление строк и/или колонок

* всегда два участника
* join() - "джойнит" один датафрейм с другим
* union() - конкатенирует один датафрейм с другим

### Join() и их виды

* Inner joins: остаются строки, которые есть в левом и правом датафреймах
* Outer joins: остаются строки, ключи которых есть в левом или правом датафрейме
* Left outer joins: остаются строки, ключи которых есть в левом датафрейме
* Right outer joins: остаются строки, ключи которых есть в правом датафрейме
* Left semi joins: остаются только строки левого датафрейма, ключи которых есть в правом датафрейме
* Left anti joins: остаются только строки левого датафрейма, ключей которых нет в правом датафрейме
* Natural joins: выполняет объединение с помощью неявного сопоставления колонок в двух датафреймах (сопоставление по именам)
* Cross (or Cartesian) joins: каждая строка левого датафрейма объединяется с каждой строкой правого датафрейма

### Метод join() и его параметры

* leftDf.join(rightDf,joinExpression,how)
* how: строка (inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi, left_anti)
* joinExpression: выражение (например `person["graduate_program"]==graduateProgram['id']`)

В материалах разбере виды join на примерах.

### Union() и его использование

* firstDf.union(secondDf)
* конкатенация данных двух датафреймов
* работает по "локации" (не по схеме)
* максимальная аккуратность, расположение колонок в нужном порядке
* unionByName() работает по именам колонок

### Lazy evaluation и оптимизация

* трансформации - это правило формирования dataframe
* "промежуточные" датафреймы в цепочке преобразований не снижают ее эффективность
* "многостраничные" select-ы появляются из-за стремления к повышению эффективности SQL запроса
* используя spark мы можем "многостраничный" select выразить последовательностью простых транформаций
* повышается "читабельность" и наглядность
* inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi, and left_antiвозможность более "гибкого" управления кэшированием промежуточных результатов

(см. примеры в материалах)

In [1]:
import os
from pyspark.sql import SparkSession

os.environ["SPARK_HOME"] = "/home/mk/mk_win/projects/SparkEdu/lib/python3.5/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

In [2]:
master = "local"
#master = "yarn"
spark = SparkSession.builder.master(master).appName("spark_test").getOrCreate()

**Специально подобранные данные**

Для того, чтобы увидеть разницу, нужно воспользоваться специально подобранными данными (иначе просто потеряться). 

Данные взяты из книги - авторы постарались...

In [20]:
person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

In [22]:
person.show()
graduateProgram.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



### Примеры join()

Выражение, по которому будет происходить join, всегда одинаково.

In [7]:
joinExpression = person["graduate_program"] == graduateProgram['id']

**Inner**

In [8]:
joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



**Outer**

появляются строки из правого фрейма, для которых нет ключей в левом

In [9]:
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



**Left outer**

поменяли местами датафреймы, теперь все строки из левого датафрейма показаны, не для всех нашлись ключи в правом

In [11]:
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



**Right outer**

ключи из правого датафрейма, не для всех нашлось соответствие в левом

In [12]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



**Left semi**

опять поменяли местами датафреймы. Остались только те программы (левый датафрейм), для которых были соответствия в правом. Правого датафрейма нет (т.е. чисто работа со строками одного датафрейма).

In [13]:
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



**Left anti**

опять поменяли местами датафреймы. Остались только те программы (левый датафрейм), для которых не было соответствия в правом. Правого датафрейма нет (т.е. чисто работа со строками одного датафрейма).

In [14]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



### Примеры union()

Просто добавим датафрейм в "хвост" себе же

In [16]:
graduateProgram.union(graduateProgram).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



Поменяем местами колонки в добавляемом датафрейме - увидим, какая "каша" получилась.

In [18]:
graduateProgram.union(graduateProgram.select("degree","id","department","school")).show()

+-------+-------+--------------------+-----------+
|     id| degree|          department|     school|
+-------+-------+--------------------+-----------+
|      0|Masters|School of Informa...|UC Berkeley|
|      2|Masters|                EECS|UC Berkeley|
|      1|  Ph.D.|                EECS|UC Berkeley|
|Masters|      0|School of Informa...|UC Berkeley|
|Masters|      2|                EECS|UC Berkeley|
|  Ph.D.|      1|                EECS|UC Berkeley|
+-------+-------+--------------------+-----------+



Если воспользоваться методом `unionByName`, то все будет корректно даже с переставленными колонками.

In [19]:
graduateProgram.unionByName(graduateProgram.select("degree","id","department","school")).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



### Многостраничный select и spark

В этой части разберем на примере - как использование spark снижает требования к SQL и позволяет создавать более модульные, "читабельные" и потенциально более эффективные программы.

Код, приведенный ниже, нельзя исполнить (нет таблиц), его можно просто посмотреть (поверив, что он работает).

Возьмем относительно простой select: 

`select
    con.contract_id as contract_id,
    con.date_insert as date_insert,
    con.date_sign as date_sign,
    con.begin_date as begin_date,
    con.end_date as end_date,
    con.product_id as product_id,
    con.channel_sale_id as channel_sale_id,
    con.contract_prev_id as contract_prev_id,
    con.contract_option_id as contract_option_id,
    con.owner_subject_id as owner_subject_id,
    con.contract_type_id as contract_type_id,
    cst.contract_status_type_id as status_type_id,
    sbj.subject_type_id as subject_type_id,
    sbj.subject_name as subject_name,
    pp.birth_date as birth_date,
    uv.hid_party as hid_party
from
    kasko.contract con
    join kasko.contract_status cst on cst.contract_status_id = con.contract_status_id
    join kasko.subject sbj on sbj.subject_id = con.owner_subject_id
    left join kasko.physical_person pp on pp.subject_id = con.owner_subject_id
    left join kasko.united_view uv on uv.source_id = sbj.subject_id and uv.source_name = 'UNI'
where
    department_id=3075`

И посмотрим, на какие "кирпичики" его можно разложить в spark.

Видно, что участвуют 5 таблиц. Начнем с того, что сформулируем, что мы хотим получить от каждой таблицы (воспользуемся синтаксисом SQL для простоты - про него чуть позже в этом модуле).

In [ ]:
# таблица contract
qCon = """
select
    contract_id as con_contract_id,
    date_insert as con_date_insert,
    date_sign as con_date_sign,
    begin_date as con_begin_date,
    end_date as con_end_date,
    product_id as con_product_id,
    channel_sale_id as con_channel_sale_id,
    contract_prev_id as con_contract_prev_id,
    contract_option_id as con_contract_option_id,
    owner_subject_id as con_owner_subject_id,
    contract_status_id as con_contract_status_id,
    contract_type_id as con_contract_type_id
from
    kasko.contract
where
    department_id = 3075
"""
dfCon = sp.sql(qCon)

In [ ]:
# таблица contract_status
qCStat = """
select 
    contract_status_id as cst_status_id,
    contract_status_type_id as cst_status_type_id
from
    kasko.contract_status
"""
dfCStat = sp.sql(qCStat)

In [ ]:
# таблица subject
qSubj = """
select
    subject_id as sbj_subject_id,
    subject_type_id as sbj_subject_type_id,
    subject_name as sbj_subject_name
from
    kasko.subject
"""
dfSubj = sp.sql(qSubj)

In [ ]:
# таблица physical_person
qPPers = """
select
    subject_id as pp_subject_id,
    birth_date as pp_birth_date
from
    kasko.physical_person
"""
dfPPers = sp.sql(qPPers)

In [ ]:
# таблица united_view
qUView = """
select
    source_id as uv_source_id,
    hid_party as uv_hid_party
from
    kasko.united_view
where
    source_name = 'UNI'
"""
dfUView = sp.sql(qUView)

Все эти датафреймы нам могут понадобиться многократно (в нашем конвейере преобразований данных), можно включить в них побольше колонок (на все случаи жизни). Получим "универсальные" кирпичики, из которых потом будем складывать "запросы" (трансформации).

Далее формулируем правила связывания датафреймов, это всего лишь выражения, которые показывают - как связаны между собой таблицы. Эти знания универсальны, могут быть унаследованы из схемы реляционной базы данных. Тоже универсальные "кирпичики" другого рода - связи между таблицами.

In [ ]:
con_stat = f.col("cst_status_id")==f.col("con_contract_status_id")
con_subj_own = f.col("con_owner_subject_id")==f.col("sbj_subject_id")
con_ppers_own = f.col("con_owner_subject_id")==f.col("pp_subject_id")
subj_uview = f.col("sbj_subject_id")==f.col("uv_source_id")

И, наконец, формулируем наш "многостраничный" запрос

In [ ]:
resDf = dfCon.join(dfSubj,con_subj_own,"inner")\
    .join(dfCStat,con_stat, "inner")\
    .join(dfPPers,con_ppers_own, "left")\
    .join(dfUView,subj_uview,"left")

Он эквивалентен исходному `select`-у, но... проще (может быть, непривычнее, но проще). Он состоит из кирпичиков, которые объединяются в простую цепочку. Оптимизатор spark сделает свое дело - итоговый запрос будет выполнен оптимально (также, как оптимально будет выполнен и исходный SQL запрос).

А теперь представим себе, что таблица `contract` у нас участвует в половине преобразований. Понять из набора select-ов, какой набор колонок нужно выбирать, какую таблицу кэшировать сложнее, чем в цепочках spark, потому что они набираются из "кирпичиков", каждый из которых фактически представляет собой подмножество полей исходных таблиц.

In [4]:
spark.stop()